In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

import os
from skimage.transform import resize
from sklearn.preprocessing import MinMaxScaler

from scipy.io import savemat, loadmat

In [ ]:
sec2year   = 365.25 * 24 * 60 * 60
psi2pascal = 6894.76
co2_rho    = 686.5266
mega       = 1e6

In [ ]:
Jfolder = '/mnt/e/Placement-Control-Optim-CO2/Johansen/data'

actnum = np.loadtxt('{}/NPD5_ActiveCells.txt'.format(Jfolder), skiprows=1).flatten().reshape(100,100,11, order='F')
actnum_pad = np.pad(actnum, ((14,14),(14,14),(2,3)), mode='constant', constant_values=0)

actnum_l1 = actnum[...,0]
indexMap = np.argwhere(actnum_l1.flatten(order='F')).flatten(order='F')

In [ ]:
rock = loadmat('data_100_100_11/rock/rock_131.mat')
poro = np.rot90(rock['poro'], 3)
perm = np.rot90(rock['perm'], 3)
print('poro: {} | perm: {}'.format(poro.shape, perm.shape))

fig, axs = plt.subplots(2, 11, figsize=(20,4), sharex=True, sharey=True)
for j in range(11):
    p = np.flip(np.ma.masked_where(actnum_l1==0, poro[:,:,j]),1)
    k = np.flip(np.ma.masked_where(actnum_l1==0, perm[:,:,j]),1)
    im0 = axs[0,j].imshow(p, cmap='jet')
    im1 = axs[1,j].imshow(k, cmap='jet')
    [plt.colorbar(i, pad=0.04, fraction=0.046) for i in [im0, im1]]
    axs[0,j].set_title('Layer {}'.format(j+1))
plt.tight_layout()
plt.show()

In [ ]:
wc = np.array(list(loadmat('data_100_100_11/well_coords/well_coords_131.mat', simplify_cells=True)['var'].values()))

tops = loadmat('data_100_100_11/Gt_cells_z.mat', simplify_cells=True)['gtz']
tops_grid = np.zeros((100,100))
tops_grid_f = tops_grid.flatten(order='F')
tops_grid_f[indexMap] = tops
tops_grid = tops_grid_f.reshape(100,100,order='F')
tops_grid_masked = np.ma.masked_where(actnum_l1==0, tops_grid)

plt.scatter(wc[1,:], wc[0,:], color='k', marker='v', s=50)
[plt.text(wc[1,i]+9, wc[0,i], 'Inj{}'.format(i+1), fontsize=12, color='k', ha='right') for i in range(wc.shape[1])]
plt.imshow(tops_grid_masked, cmap='jet_r', interpolation='none')
plt.colorbar()
plt.tight_layout()
plt.show()

In [ ]:
n_realizations = 3
timesteps = []
saturation_f = np.zeros((n_realizations,31,29128))
pressure_f   = np.zeros((n_realizations,31,29128))

sec2year = 365.25 * 24 * 60 * 60
psi2pascal = 6894.76

for t in range(1,30):
    d = loadmat('data_100_100_11/states/states_0.mat', simplify_cells=True)['var']
    timesteps.append(d[t]['time']/sec2year)
timesteps = [0] + timesteps

for i in range(n_realizations):
    d = loadmat('data_100_100_11/states/states_{}.mat'.format(i), simplify_cells=True)['var']
    for t in range(len(d)):
        saturation_f[i,t] = d[t]['s'][:,1]
        pressure_f[i,t]   = d[t]['pressure']

In [ ]:
indexMap = loadmat('data_100_100_11/G_cells_indexMap.mat', simplify_cells=True)['G_cells_indexMap']

In [ ]:
saturation = np.zeros((n_realizations, len(timesteps), 100*100*11))
pressure   = np.zeros((n_realizations, len(timesteps), 100*100*11))

for i in range(n_realizations):
    for t in range(len(timesteps)):
        saturation[i,t][indexMap] = saturation_f[i,t][::-1]
        pressure[i,t][indexMap]   = pressure_f[i,t] / psi2pascal

saturation = saturation.reshape(n_realizations, len(timesteps), 11, 100, 100)
pressure   = pressure.reshape(n_realizations, len(timesteps), 11, 100, 100)
print(saturation.shape, pressure.shape)

In [ ]:
well_rates = {}
for i in range(n_realizations):
    d = loadmat('data_100_100_11/VE_states/states_{}.mat'.format(i), simplify_cells=True)['var']
    d0 = d[1]['wellSol']
    nw = 1 if len(d0) > 3 else len(d0)
    wr = np.zeros((len(timesteps), nw))
    for w in range(nw):
        for t in range(1,len(timesteps)):
            wr[t,w] = d[t]['wellSol'][w]['val'] if nw > 1 else  d[t]['wellSol']['val']
    well_rates[i] = wr * co2_rho * sec2year / 1e3 / mega

In [ ]:
d131 = loadmat('data_100_100_11/states/states_131.mat', simplify_cells=True)['var']

timesteps = []
for i in range(39):
    t = d131[i]['time']
    timesteps.append(t/sec2year)

d0 = d131[1]['wellSol']
nw = 1 if len(d0) > 3 else len(d0)
wr = np.zeros((len(timesteps), nw))
for w in range(nw):
    for t in range(1,len(timesteps)):
        wr[t,w] = d131[t]['wellSol'][w]['val'] if nw > 1 else  d131[t]['wellSol']['val']

In [ ]:
fig, axs = plt.subplots(1, wr.shape[1], figsize=(12,4), sharex=True, sharey=True)
for w in range(wr.shape[1]):
    ax = axs[w]
    ax.set(xlabel='Time [years]', xticks=range(11), xticklabels=range(11), xlim=(0,10), title='Injector {}'.format(w+1))
    ax.set_ylabel('Injection rate [Mt CO2/year]', c='r')
    ax.grid(True, which='both', alpha=0.5)
    ax2 = ax.twinx()
    ax2.plot(timesteps[:29], np.cumsum(wr[:29,w]*timesteps[:29]), color='b', marker='.')
    ax2.set_ylabel('Cumulative CO2 Injected [Mt]', color='b', rotation=270, labelpad=15)
    for t in range(1,29):
        ax.plot([timesteps[t-1], timesteps[t]], [wr[t,w], wr[t,w]], 'r')
        ax.vlines(timesteps[t], wr[t,w], wr[t+1,w], ls='--', color='k')    
plt.tight_layout()
plt.show()

In [ ]:
sat0 = np.zeros((100*100*11))
pres0 = np.zeros((100*100*11))

sat0[indexMap] = np.ma.masked_where(indexMap, d131[-1]['s'][:,1])
s0 = np.moveaxis(sat0.reshape(11,100,100), 0, -1)
s0 = np.ma.masked_where(actnum==0, s0)

pres0[indexMap] = np.ma.masked_where(indexMap, d131[-1]['pressure'])
p0 = np.moveaxis(pres0.reshape(11,100,100), 0, -1)
p0 = np.ma.masked_where(actnum==0, p0)

plt.figure(figsize=(20,6))
for i in range(11):
    plt.subplot(1, 11, i+1)
    plt.imshow(s0[...,i], cmap='jet', vmin=0, vmax=0.7)
    plt.scatter(wc[0,:], wc[1,:], color='k', marker='v', s=25)
plt.tight_layout()
plt.show()

plt.figure(figsize=(20,6))
for i in range(11):
    plt.subplot(1, 11, i+1)
    plt.imshow(p0[...,i], cmap='jet_r')
    plt.scatter(wc[0,:], wc[1,:], color='k', marker='v', s=25)
plt.tight_layout()
plt.show()

In [ ]:
realization = 2

plt.figure(figsize=(20,7))
k = 0
for i in range(3):
    for j in range(10):
        plt.subplot(3,10,k+1)
        sat_masked = np.ma.masked_where(actnum_l1==0, saturation[realization,k,5])
        im = plt.imshow(sat_masked, cmap='jet', vmin=0, vmax=0.25)
        plt.title('t={:.1f}'.format(round(timesteps[k], 2)))
        plt.colorbar(im, pad=0.04, fraction=0.046)
        plt.xticks([]); plt.yticks([])
        k += 1
plt.tight_layout()
plt.show()

In [ ]:
plt.figure(figsize=(20,7))
k = 0
for i in range(4):
    for j in range(10):
        plt.subplot(4,10,k+1)
        k += 1
        pres_masked = np.ma.masked_where(actnum_l1==0, pressure[realization,k])
        im = plt.imshow(pres_masked, cmap='jet')
        plt.title('t={:.1f}'.format(round(timesteps[k], 2)))
        plt.colorbar(im, pad=0.04, fraction=0.046)
        plt.xticks([]); plt.yticks([])
plt.tight_layout()
plt.show()

In [ ]:
fig, axs = plt.subplots(1, well_rates[realization].shape[1], figsize=(12,4), sharex=True, sharey=True)
for w in range(well_rates[realization].shape[1]):
    ax = axs[w]
    ax.set(xlabel='Time [years]', xticks=range(11), xticklabels=range(11), xlim=(0,10), title='Injector {}'.format(w+1))
    ax.set_ylabel('Injection rate [Mt CO2/year]', c='r')
    ax.grid(True, which='both', alpha=0.5)
    ax2 = ax.twinx()
    ax2.plot(timesteps[:20], np.cumsum(well_rates[realization][:20,w]*timesteps[:20]), color='b', marker='.')
    ax2.set_ylabel('Cumulative CO2 Injected [Mt]', color='b', rotation=270, labelpad=15)
    for t in range(1,21):
        ax.plot([timesteps[t-1], timesteps[t]], [well_rates[realization][t,w], well_rates[realization][t,w]], 'r')
        ax.vlines(timesteps[t], well_rates[realization][t,w], well_rates[realization][t+1,w], ls='--', color='k')    
plt.tight_layout()
plt.show()

In [ ]:
report = loadmat('data_100_100_11/VE_reports_NoFlowBC/reports_21.mat', simplify_cells=True)['var']
sec2year = 365.25 * 24 * 60 * 60
mt_of_co2 = 686.5266 * 1e6

t = []
for i in range(31):
    t.append(report[i]['t']/sec2year)

m = np.zeros((31,8))
for i in range(31):
    for j in range(8):
        m[i,j] = report[i]['masses'][j] / mt_of_co2

In [ ]:
colors = ['k','blue','green','lightgreen','gold','c','orange','r']
labels = ['dissolved','structural residual', 'residual','residual in plume','structural plume', 'structural plume (t)','free plume', 'exited']

for i in range(8):
    plt.plot(t, np.cumsum(m, axis=1), color='k', lw=0.25)
    plt.fill_between(t, np.cumsum(m, axis=1)[:,i], np.cumsum(m, axis=1)[:,i-1], label=labels[i], color=colors[i])

plt.vlines(10, 0, np.cumsum(m,1).max(), color='k', ls='--')
plt.text(12, 30, 'end of injection', bbox=dict(facecolor='lightgray', edgecolor='k'))

plt.xscale('log')
plt.xlim(0.5, 510)
plt.legend(bbox_to_anchor=(1.015, 0.5), loc='center left', facecolor='lightgrey', edgecolor='k')
plt.xlabel('Time [years]'); plt.ylabel('Mass CO2 [Mt]')
plt.grid(True, which='both', alpha=0.33)
plt.show()